In [14]:
!pip install langchain

In [ ]:
pip install --upgrade langchain

In [4]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [7]:
conda install -c pyviz panel


Note: you may need to restart the kernel to use updated packages.


usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: panel


In [3]:
import nltk
nltk.download('punkt')

E:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sendt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
 pip install panel

In [9]:
pip install pypdf

In [11]:
pip install -U langchain-openai

  Using cached langchain_core-0.1.32-py3-none-any.whl (260 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.92
    Uninstalling langsmith-0.0.92:
      Successfully uninstalled langsmith-0.0.92
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.0.13
    Uninstalling langchain-core-0.0.13:
      Successfully uninstalled langchain-core-0.0.13
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.0.346 requires langchain-core<0.1,>=0.0.10, but you have langchain-core 0.1.32 which is incompatible.
langchain 0.0.346 requires langsmith<0.1.0,>=0.0.63, but you have langsmith 0.1.27 which is incompatible.


In [12]:
pip install Pillow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# ChatBot

In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk  # Import PIL library for image handling
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from datetime import datetime
import os

os.environ["OPENAI_API_KEY"] = "sk-"

class PDFChatbotGUI:
    def __init__(self, master, openai_api_key):
        self.master = master
        self.master.title("T Chatbot")
        self.master.geometry("600x500")
        self.openai_api_key = openai_api_key
        self.selected_files = []

        # Styling
        self.bg_color = "#f0f0f0"
        self.button_color = "#4CAF50"
        self.button_fg_color = "white"
        self.text_color = "#333333"
        #self.logo_path = "trafikverket_logo.png"  # Path to the Trafikverket logo image

        # Labels
        self.file_path_label = tk.Label(master, text="PDF File Path:", bg=self.bg_color, fg=self.text_color, font=("Helvetica", 12, "bold"))
        self.file_path_label.grid(row=0, column=0, pady=10, padx=20, sticky="w")

        # Upload Button
        self.upload_button = tk.Button(master, text="Upload Files", command=self.upload_files, bg=self.button_color, fg=self.button_fg_color, font=("Helvetica", 10, "bold"), relief="raised", bd=3)
        self.upload_button.grid(row=0, column=1, pady=10, padx=20)

        # Question Label
        self.question_label = tk.Label(master, text="Enter your question:", bg=self.bg_color, fg=self.text_color, font=("Helvetica", 12, "bold"))
        self.question_label.grid(row=1, column=0, pady=10, padx=20, sticky="w")

        # Question Entry
        self.question_entry = tk.Entry(master, width=50, bg="white", font=("Helvetica", 10))
        self.question_entry.grid(row=1, column=1, pady=10, padx=20)

        # Chat Text
        self.chat_text = tk.Text(master, height=15, width=70, wrap="word", bg="white", font=("Helvetica", 10))
        self.chat_text.grid(row=3, column=0, columnspan=2, pady=20, padx=20, sticky="nsew")
        self.chat_text.tag_configure("highlight", background="#80DEEA")  # Light blue highlight color

        # Loading Button
        self.loading_button = tk.Button(master, text="Loading...", state="disabled", bg=self.button_color, fg=self.button_fg_color, font=("Helvetica", 10, "bold"), relief="raised", bd=3)
        self.loading_button.grid(row=4, column=0, columnspan=2, pady=10, padx=20, sticky="we")  # Initially hidden

        # Ask Button
        self.ask_button = tk.Button(master, text="Ask", command=self.ask_question, bg=self.button_color, fg=self.button_fg_color, font=("Helvetica", 10, "bold"), relief="raised", bd=3)
        self.ask_button.grid(row=2, column=1, pady=10, padx=20)
        self.ask_button.grid_remove()  # Initially hidden

        # Load and display the Trafikverket logo in the chat text background
        #self.load_logo()

        # Initialize attributes
        self.embeddings = None
        self.new_model = 'gpt-3.5-turbo-instruct'
        self.chain = None


    def upload_files(self):
        self.show_loading_button()
        self.file_names = []
        self.raw_texts = []
        self.document_searches = []
        files = filedialog.askopenfilenames(filetypes=[("PDF Files", "*.pdf")])
        for file_path in files:
            try:
                self.file_names.append(os.path.basename(file_path))
                pdf_reader = PdfReader(file_path)
                raw_text = ''
                for page in pdf_reader.pages:
                    content = page.extract_text()
                    if content:
                        raw_text += content
                self.raw_texts.append(raw_text)
                print("Uploaded:", file_path)
            except Exception as e:
                messagebox.showerror("Error", f"An error occurred while processing the PDF file '{file}': {e}")
        self.process_pdf()

    def process_pdf(self):
        self.show_loading_button()
        self.document_searches = []
        for raw_text in self.raw_texts:
            if raw_text:
                text_splitter = CharacterTextSplitter(
                    separator="\n",
                    chunk_size=800,
                    chunk_overlap=200,
                    length_function=len,
                )
                texts = text_splitter.split_text(raw_text)
                embeddings = OpenAIEmbeddings()
                document_search = FAISS.from_texts(texts, embeddings)
                self.document_searches.append(document_search)
            self.load_qa_chain()

    def load_qa_chain(self):
        try:
            if not self.document_searches:  # If document_searches is empty
                messagebox.showerror("Error", "No PDF documents loaded.")
                return
            self.chain = load_qa_chain(OpenAI(api_key=self.openai_api_key), chain_type="stuff")
            self.ask_button.grid()  # Display the Ask button
            self.chat_text.insert(tk.END, "Chatbot: PDF loaded successfully. You can start chatting.\n")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred while loading the question answering chain: {e}")

    def ask_question(self):
        question = self.question_entry.get()  # Fetch question from entry widget
        if not question:
            messagebox.showerror("Error", "Please enter a question.")
            return
        try:
            self.show_loading_button()
            all_responses = []
            for file_name, doc_search in zip(self.file_names, self.document_searches):
                docs = doc_search.similarity_search(question)
                response = self.chain.run(input_documents=docs, question=question)
                all_responses.append((file_name, response))
            for file_name, response in all_responses:
                self.chat_text.insert(tk.END, f"File {file_name}: {response}\n")
            self.chat_text.insert(tk.END, f"You: {question}\n")

            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            with open("question_answers.txt", "a") as file:
                file.write(f"Time: {current_time}\n")
                file.write(f"Question: {question}\n")
                for file_name, response in all_responses:
                    file.write(f"File {file_name}: {response}\n")
                file.write("\n")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred during question answering: {e}")
        finally:
            self.hide_loading_button()

    def show_loading_button(self):
        self.ask_button.grid_remove()  # Hide Ask button
        self.loading_button.grid(row=4, column=0, columnspan=2)  # Display loading button

    def hide_loading_button(self):
        self.loading_button.grid_remove()  # Hide loading button
        self.ask_button.grid()  # Display Ask button

def main():
    root = tk.Tk()
    openai_api_key = "sk"
    app = PDFChatbotGUI(root, openai_api_key)
    root.mainloop()

if __name__ == "__main__":
    main()


Uploaded: D:/dalarna research/TDOK_2010-50.pdf
Uploaded: D:/dalarna research/TDOK_2010-120.pdf
Uploaded: D:/dalarna research/TDOK_2010-202.pdf
Uploaded: D:/dalarna research/TDOK_2015-0430.pdf
Uploaded: D:/dalarna research/TDOK_2016-0077.pdf
Uploaded: D:/dalarna research/TDOK_2018-0371.pdf
Uploaded: D:/dalarna research/TDOK_2020-0216.pdf
